In [1]:
import pandas as pd

In [2]:
events_df = pd.read_csv("./Dirigo/OCEL_Events.csv")
e2o_df = pd.read_csv("./Dirigo/OCEL_E2O.csv")

In [3]:
prefix_map = {
    'Pcp': 'PickupPlan',
    'tr': 'Truck',
    'Cr': 'Cargo'
}
e2o_df['prefix'] = e2o_df['Object_id'].str.extract(r'^([A-Za-z]+)')
e2o_df['object_type'] = e2o_df['prefix'].map(prefix_map)


In [4]:
events_with_objects = events_df.copy()
object_types = e2o_df['object_type'].dropna().unique()
for obj_type in object_types:
    related = e2o_df[e2o_df['object_type'] == obj_type]
    grouped = related.groupby('Event_id')['Object_id'].apply(list).reset_index()
    grouped.rename(columns={'Object_id': obj_type}, inplace=True)
    events_with_objects = events_with_objects.merge(grouped, how='left', on='Event_id')

In [5]:
events_with_objects.to_csv("./DOCEL/DOCEL_Events.csv", index=False)

In [6]:
truck_df = pd.read_csv("./Dirigo/OCEL_Truck.csv")

In [7]:
# truck_df.rename(columns={
#     'PickupPlanID': 'Pickup Plan ID',
#     'CargoID': 'Cargo ID'
# }, inplace=True)

In [8]:
def extract_static_objects_strict(static_df):
  
    dynamic_attrs = set(static_df["Ocel_changed_field"].dropna().unique())

 
    exclude_cols = dynamic_attrs.union({"Ocel_changed_field", "Timestamp"})
    static_cols = [col for col in static_df.columns if col not in exclude_cols]


    static_objects_df = static_df[static_cols].drop_duplicates(subset=["Object_id"]).copy()


    static_objects_df = static_objects_df.rename(columns={"Object_id": "ObjectId"})

    return static_objects_df

In [9]:
docel_truck = extract_static_objects_strict(truck_df)
docel_truck.rename(columns={'ObjectId': 'Truck ID'}, inplace=True)
docel_truck

,Truck ID,LPT,Axles
0,tr46,271FSE,6.0
1,tr28,157LCX,4.0
2,tr5,012CRA,6.0
3,tr27,764KIO,4.0
4,tr41,055DSW,4.0
5,tr12,710YJQ,2.0
6,tr36,671NRX,4.0
7,tr37,280QHM,6.0
8,tr33,823MWT,6.0
9,tr49,606MXL,6.0


In [10]:
docel_truck.to_csv("./DOCEL/DOCEL_Truck_Static.csv", index=False)

In [11]:
cargo_df = pd.read_csv("./Dirigo/OCEL_Cargo.csv")

In [12]:
docel_cargo = extract_static_objects_strict(cargo_df)
docel_cargo.rename(columns={'ObjectId': 'Cargo ID'}, inplace=True)
docel_cargo

,Cargo ID,Cargo Type,Silo ID
0,Cr20,Corn,Sil4
1,Cr15,Rice,Sil15
2,Cr7,Rice,Sil1
3,Cr19,Other,Sil13
4,Cr3,Wheat,Sil10
5,Cr10,Corn,Sil16
6,Cr8,Rice,Sil2
7,Cr18,Other,Sil11
8,Cr14,Rice,Sil17
9,Cr11,Other,Sil19


In [13]:
docel_cargo.to_csv("./DOCEL/DOCEL_Cargo_Static.csv", index=False)

In [14]:
def extract_dynamic_attribute_names(df):
    return sorted(set(df["Ocel_changed_field"].dropna().unique()))

In [15]:
truck_dynamic = extract_dynamic_attribute_names(truck_df)
truck_dynamic

['Cargo ID',
 'Pickup Plan ID',
 'Scheduled Pickup Weight',
 'Truck Status',
 'Truck Weight']

In [16]:
def generate_dynamic_attribute_table(object_df, attr_name, e2o_df, events_df):
  
    object_df = object_df.copy()
    object_df['Timestamp'] = pd.to_datetime(object_df['Timestamp'])
    events_df['Timestamp'] = pd.to_datetime(events_df['Timestamp'])

    e2o_full = e2o_df.merge(events_df, on='Event_id', how='left')
    e2o_full.rename(columns={'Timestamp': 'event_timestamp'}, inplace=True)

    target_rows = object_df[object_df['Ocel_changed_field'] == attr_name].copy()

    records = []
    for idx, row in target_rows.iterrows():
        object_id = row['Object_id']
        timestamp = row['Timestamp']
        value = row.get(attr_name)

        if pd.isna(object_id) or pd.isna(value) or pd.isna(timestamp):
            continue

        related_events = e2o_full[(e2o_full['Object_id'] == object_id) &
                                  (e2o_full['event_timestamp'] == timestamp)]

        if related_events.empty:
            continue  

        event_id = related_events.iloc[0]['Event_id']

        records.append({
            'ID': f"{attr_name}_{len(records)}",
            'value': value,
            'objectID': object_id,
            'eventID': event_id
        })

    return pd.DataFrame(records)


In [17]:
TW_df = generate_dynamic_attribute_table(truck_df, "Truck Weight", e2o_df, events_df)
TW_df.rename(columns={'ID': 'TWID','value':'Truck Weight'}, inplace=True)
TW_df.to_csv('./DOCEL/DOCEL_TruckWeight.csv',index=False)

In [18]:
TS_df = generate_dynamic_attribute_table(truck_df, "Truck Status", e2o_df, events_df)
TS_df.rename(columns={'ID': 'TSID','value':'Truck Status'}, inplace=True)
TS_df

,TSID,Truck Status,objectID,eventID
0,Truck Status_0,Occupied,tr26,assign_tr26_Pcp44
1,Truck Status_1,Occupied,tr49,assign_tr49_Pcp44
2,Truck Status_2,Occupied,tr41,assign_tr41_Pcp87
3,Truck Status_3,Occupied,tr22,assign_tr22_Pcp87
4,Truck Status_4,Occupied,tr11,assign_tr11_Pcp48
...,...,...,...,...
195,Truck Status_195,Occupied,tr17,assign_tr17_Pcp77
196,Truck Status_196,Occupied,tr23,assign_tr23_Pcp51
197,Truck Status_197,Occupied,tr29,assign_tr29_Pcp51
198,Truck Status_198,Occupied,tr29,assign_tr29_Pcp70


In [19]:
TS_df.to_csv('./DOCEL/DOCEL_TruckStatus.csv',index=False)

In [20]:
TCargo_df = generate_dynamic_attribute_table(truck_df, "Cargo ID", e2o_df, events_df)
TCargo_df.rename(columns={'ID': 'TCrID','value':'Cargo ID'}, inplace=True)
TCargo_df

,TCrID,Cargo ID,objectID,eventID
0,Cargo ID_0,Cr2,tr26,assign_tr26_Pcp44
1,Cargo ID_1,Cr2,tr49,assign_tr49_Pcp44
2,Cargo ID_2,Cr9,tr41,assign_tr41_Pcp87
3,Cargo ID_3,Cr9,tr22,assign_tr22_Pcp87
4,Cargo ID_4,Cr20,tr11,assign_tr11_Pcp48
...,...,...,...,...
195,Cargo ID_195,Cr13,tr17,assign_tr17_Pcp77
196,Cargo ID_196,Cr18,tr23,assign_tr23_Pcp51
197,Cargo ID_197,Cr18,tr29,assign_tr29_Pcp51
198,Cargo ID_198,Cr10,tr29,assign_tr29_Pcp70


In [21]:
TCargo_df.to_csv('./DOCEL/DOCEL_TruckCargoID.csv',index=False)

In [22]:
TPcp_df = generate_dynamic_attribute_table(truck_df, "Pickup Plan ID", e2o_df, events_df)
TPcp_df.rename(columns={'ID': 'TPcpID','value':'Pickup Plan ID'}, inplace=True)
TPcp_df

,TPcpID,Pickup Plan ID,objectID,eventID
0,Pickup Plan ID_0,Pcp44,tr26,assign_tr26_Pcp44
1,Pickup Plan ID_1,Pcp44,tr49,assign_tr49_Pcp44
2,Pickup Plan ID_2,Pcp87,tr41,assign_tr41_Pcp87
3,Pickup Plan ID_3,Pcp87,tr22,assign_tr22_Pcp87
4,Pickup Plan ID_4,Pcp48,tr11,assign_tr11_Pcp48
...,...,...,...,...
195,Pickup Plan ID_195,Pcp77,tr17,assign_tr17_Pcp77
196,Pickup Plan ID_196,Pcp51,tr23,assign_tr23_Pcp51
197,Pickup Plan ID_197,Pcp51,tr29,assign_tr29_Pcp51
198,Pickup Plan ID_198,Pcp70,tr29,assign_tr29_Pcp70


In [23]:
TPcp_df.to_csv('./DOCEL/DOCEL_TruckPickupplanID.csv',index=False)

In [24]:
TSchW_df = generate_dynamic_attribute_table(truck_df, "Scheduled Pickup Weight", e2o_df, events_df)
TSchW_df.rename(columns={'ID': 'TSchWID','value':'Scheduled Pickup Weight'}, inplace=True)
TSchW_df

,TSchWID,Scheduled Pickup Weight,objectID,eventID
0,Scheduled Pickup Weight_0,5369.5,tr26,assign_tr26_Pcp44
1,Scheduled Pickup Weight_1,5241.8,tr49,assign_tr49_Pcp44
2,Scheduled Pickup Weight_2,5305.6,tr41,assign_tr41_Pcp87
3,Scheduled Pickup Weight_3,5509.5,tr22,assign_tr22_Pcp87
4,Scheduled Pickup Weight_4,5476.2,tr11,assign_tr11_Pcp48
...,...,...,...,...
195,Scheduled Pickup Weight_195,5283.8,tr17,assign_tr17_Pcp77
196,Scheduled Pickup Weight_196,4941.8,tr23,assign_tr23_Pcp51
197,Scheduled Pickup Weight_197,5063.2,tr29,assign_tr29_Pcp51
198,Scheduled Pickup Weight_198,5458.8,tr29,assign_tr29_Pcp70


In [25]:
TSchW_df.to_csv('./DOCEL/DOCEL_TruckScheduledPickupWeight.csv',index=False)

In [26]:
cargo_dynamic = extract_dynamic_attribute_names(cargo_df)
cargo_dynamic

['Cargo stock weight(scheduled)']

In [27]:
CargoSt_df = generate_dynamic_attribute_table(cargo_df, "Cargo stock weight(scheduled)", e2o_df, events_df)
CargoSt_df.rename(columns={'ID': 'CargoStID','value':'Cargo stock weight(scheduled)'}, inplace=True)
CargoSt_df

,CargoStID,Cargo stock weight(scheduled),objectID,eventID
0,Cargo stock weight(scheduled)_0,90032.0,Cr2,Lodge_Pcp44
1,Cargo stock weight(scheduled)_1,133114.7,Cr9,Lodge_Pcp87
2,Cargo stock weight(scheduled)_2,112866.2,Cr20,Lodge_Pcp48
3,Cargo stock weight(scheduled)_3,137342.6,Cr13,Lodge_Pcp55
4,Cargo stock weight(scheduled)_4,130779.5,Cr7,Lodge_Pcp69
...,...,...,...,...
95,Cargo stock weight(scheduled)_95,70689.5,Cr20,Lodge_Pcp37
96,Cargo stock weight(scheduled)_96,55629.0,Cr14,Lodge_Pcp75
97,Cargo stock weight(scheduled)_97,39322.9,Cr15,Lodge_Pcp62
98,Cargo stock weight(scheduled)_98,77221.3,Cr17,Lodge_Pcp50


In [28]:
CargoSt_df.to_csv('./DOCEL/DOCEL_CargoStock.csv',index=False)

In [29]:
def generate_dynamic_tables_from_pickupplan(pickup_df, e2o_df, events_df):

    pickup_df = pickup_df.copy()
    pickup_df['Timestamp'] = pd.to_datetime(pickup_df['Timestamp'])
    events_df['Timestamp'] = pd.to_datetime(events_df['Timestamp'])

    e2o_full = e2o_df.merge(events_df, on='Event_id', how='left')
    e2o_full.rename(columns={'Timestamp': 'event_timestamp'}, inplace=True)

    attr_columns = [col for col in pickup_df.columns if col not in ['Object_id', 'Timestamp', 'Ocel_changed_field']]
    results = {}

    for attr in attr_columns:
        records = []

        for _, row in pickup_df.iterrows():
            object_id = row['Object_id']
            timestamp = row['Timestamp']
            value = row.get(attr)

            if pd.isna(object_id) or pd.isna(timestamp) or pd.isna(value):
                continue

            related_events = e2o_full[(e2o_full['Object_id'] == object_id) &
                                      (e2o_full['event_timestamp'] == timestamp)]
            if related_events.empty:
                continue

            event_id = related_events.iloc[0]['Event_id']


            records.append({
                'ID': 
                f"{attr}_{len(records)}",
                'value': value,
                'objectID': object_id,
                'eventID': event_id
            })

        results[attr] = pd.DataFrame(records)

    return results


In [30]:
pickup_df = pd.read_csv("./Dirigo/OCEL_PickupPlan.csv")
pickup_dynamic_tables = generate_dynamic_tables_from_pickupplan(pickup_df, e2o_df, events_df)
pickup_dynamic_tables

{'CargoID':             ID value objectID      eventID
 0    CargoID_0   Cr2    Pcp44  Lodge_Pcp44
 1    CargoID_1   Cr9    Pcp87  Lodge_Pcp87
 2    CargoID_2  Cr20    Pcp48  Lodge_Pcp48
 3    CargoID_3  Cr13    Pcp55  Lodge_Pcp55
 4    CargoID_4   Cr7    Pcp69  Lodge_Pcp69
 ..         ...   ...      ...          ...
 95  CargoID_95  Cr20    Pcp37  Lodge_Pcp37
 96  CargoID_96  Cr14    Pcp75  Lodge_Pcp75
 97  CargoID_97  Cr15    Pcp62  Lodge_Pcp62
 98  CargoID_98  Cr17    Pcp50  Lodge_Pcp50
 99  CargoID_99  Cr13    Pcp77  Lodge_Pcp77
 
 [100 rows x 4 columns],
 'Num of trucks':                   ID  value objectID      eventID
 0    Num of trucks_0      2    Pcp44  Lodge_Pcp44
 1    Num of trucks_1      2    Pcp87  Lodge_Pcp87
 2    Num of trucks_2      2    Pcp48  Lodge_Pcp48
 3    Num of trucks_3      2    Pcp55  Lodge_Pcp55
 4    Num of trucks_4      2    Pcp69  Lodge_Pcp69
 ..               ...    ...      ...          ...
 95  Num of trucks_95      2    Pcp37  Lodge_Pcp37
 96  Num 

In [31]:
CargoPP_df = pickup_dynamic_tables["CargoID"]
CargoPP_df.rename(columns={'ID': 'PcpCrID','value':'Cargo ID'}, inplace=True)
CargoPP_df

,PcpCrID,Cargo ID,objectID,eventID
0,CargoID_0,Cr2,Pcp44,Lodge_Pcp44
1,CargoID_1,Cr9,Pcp87,Lodge_Pcp87
2,CargoID_2,Cr20,Pcp48,Lodge_Pcp48
3,CargoID_3,Cr13,Pcp55,Lodge_Pcp55
4,CargoID_4,Cr7,Pcp69,Lodge_Pcp69
...,...,...,...,...
95,CargoID_95,Cr20,Pcp37,Lodge_Pcp37
96,CargoID_96,Cr14,Pcp75,Lodge_Pcp75
97,CargoID_97,Cr15,Pcp62,Lodge_Pcp62
98,CargoID_98,Cr17,Pcp50,Lodge_Pcp50


In [32]:
CargoPP_df.to_csv('./DOCEL/DOCEL_PickupplanCargo.csv',index=False)

In [33]:
Notrs_df = pickup_dynamic_tables["Num of trucks"]
Notrs_df.rename(columns={'ID': 'NotrsID','value':'Num of trucks'}, inplace=True)
Notrs_df

,NotrsID,Num of trucks,objectID,eventID
0,Num of trucks_0,2,Pcp44,Lodge_Pcp44
1,Num of trucks_1,2,Pcp87,Lodge_Pcp87
2,Num of trucks_2,2,Pcp48,Lodge_Pcp48
3,Num of trucks_3,2,Pcp55,Lodge_Pcp55
4,Num of trucks_4,2,Pcp69,Lodge_Pcp69
...,...,...,...,...
95,Num of trucks_95,2,Pcp37,Lodge_Pcp37
96,Num of trucks_96,2,Pcp75,Lodge_Pcp75
97,Num of trucks_97,2,Pcp62,Lodge_Pcp62
98,Num of trucks_98,2,Pcp50,Lodge_Pcp50


In [34]:
Notrs_df.to_csv('./DOCEL/DOCEL_PickupplanNotrs.csv',index=False)

In [35]:
PcpTotalW_df = pickup_dynamic_tables["Total pickup weight"]
PcpTotalW_df.rename(columns={'ID': 'PcpTotalWID','value':'Total pickup weight'}, inplace=True)
PcpTotalW_df

,PcpTotalWID,Total pickup weight,objectID,eventID
0,Total pickup weight_0,10611.3,Pcp44,Lodge_Pcp44
1,Total pickup weight_1,10815.1,Pcp87,Lodge_Pcp87
2,Total pickup weight_2,11155.8,Pcp48,Lodge_Pcp48
3,Total pickup weight_3,11720.1,Pcp55,Lodge_Pcp55
4,Total pickup weight_4,10058.4,Pcp69,Lodge_Pcp69
...,...,...,...,...
95,Total pickup weight_95,10831.7,Pcp37,Lodge_Pcp37
96,Total pickup weight_96,11362.6,Pcp75,Lodge_Pcp75
97,Total pickup weight_97,10123.2,Pcp62,Lodge_Pcp62
98,Total pickup weight_98,11828.4,Pcp50,Lodge_Pcp50


In [36]:
PcpTotalW_df.to_csv('./DOCEL/DOCEL_PickupplanTotalWeight.csv',index=False)